Pororo 라이브러리 설치

In [ ]:
!pip --version

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [ ]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 10.5 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [ ]:
!git clone https://github.com/kakaobrain/pororo.git

Cloning into 'pororo'...
remote: Enumerating objects: 2119, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 2119 (delta 11), reused 22 (delta 8), pack-reused 2090
Receiving objects: 100% (2119/2119), 13.09 MiB | 18.24 MiB/s, done.
Resolving deltas: 100% (1448/1448), done.


In [ ]:
ls

pororo/  sample_data/


In [ ]:
cd pororo

/content/pororo


In [ ]:
!pip install -e .
# 이 셀 끝나고 '런타임 다시 시작'

Pororo 사용하여 역번역

=============================================================================

In [ ]:
import numpy as np
import re
import pandas as pd
import json
from collections import defaultdict
from pororo import Pororo
from tqdm import tqdm

In [ ]:
# 데이터 불러오기
data_train = [json.loads(line) for line in open('', 'r')]

In [ ]:
# 중복 키값을 위한 class 설정
class key_name(object):
  def __init__(self, name):
    self.name = name
  
  def __str__(self):
    return self.name

  def __repr__(self):
    return "'"+self.name+"'"

In [ ]:
# data_train 형식에 따라 data_trian[0] 또는 data_train 사용하기에 여기서 바꿔줌
data_train = [data_train]

In [ ]:
# 제이슨을 딕셔너리로 바꾸기

dic_train = defaultdict(int)

for i in range(len(data_train[0])):
  for j in range(len(data_train[0][i]['annotation'])):#각 순번의 annotation 개수가 다르기에 2번째 for문 사용
    dic_train[key_name(data_train[0][i]['sentence_form'])] = data_train[0][i]['annotation'][j][0], data_train[0][i]['annotation'][j][1], data_train[0][i]['annotation'][j][2]
    # key_name을 사용해 중복 key를 사용함 >> 한문장에 하나의 annotation 만 들어갈 수 있도록

In [ ]:
# 딕셔너리를 DataFrame으로 바꾸는 과정
df = pd.DataFrame(dic_train)
# 행렬 변환
df = df.T
# 인덱스 재설정
df = df.reset_index()
# 열이름 재설정
df = df.rename(columns = {'index':'sentence', 0:'category', 1:'topic',2:'sense'})
df_train = df

In [ ]:
# Pororo 번역 task 변수 설정
mt = Pororo(task = 'translation', lang = 'multi')

In [ ]:
# 조건에 맞는 행의 인덱스 추출

mt_list = []
for i in range(len(df_train['sentence'])):
  if df_train['category'][i] == '브랜드#품질':
    mt_list.append([df_train['sentence'][i], i])
  elif df_train['category'][i] == '브랜드#디자인':
    mt_list.append([df_train['sentence'][i], i])
  elif df_train['category'][i] == '브랜드#인지도':
    mt_list.append([df_train['sentence'][i], i])
  elif df_train['category'][i] == '브랜드#가격':
    mt_list.append([df_train['sentence'][i], i])
  elif df_train['category'][i] == '제품 전체#다양성':
    mt_list.append([df_train['sentence'][i], i])
  elif df_train['category'][i] == '패키지/구성품#다양성':
    mt_list.append([df_train['sentence'][i], i])
  elif df_train['category'][i] == '패키지/구성품#가격':
    mt_list.append([df_train['sentence'][i], i])
  elif df_train['category'][i] == '패키지/구성품#일반':
    mt_list.append([df_train['sentence'][i], i])
  elif df_train['category'][i] == '패키지/구성품#품질':
    mt_list.append([df_train['sentence'][i], i])
  elif df_train['category'][i] == '본품#인지도':
    mt_list.append([df_train['sentence'][i], i])
  elif df_train['category'][i] == '본품#다양성':
    mt_list.append([df_train['sentence'][i], i])

In [ ]:
df_aug = [] # 역번역된 문장과 기존 문장의 번호를 써둘 빈 리스트 만들기

for i in tqdm(range(len(mt_list))):

  translation = mt(str(mt_list[i][0]),src = 'ko', tgt = 'en') # 한글을 영어로 번역
  translation = mt(translation,src = 'en', tgt = 'ko') # 번역된 영문을 다시 한글로 번역
  df_aug.append([translation, mt_list[i][1]]) # 역번역 완료된 문장과 역번역 적용된 기존문장의 번호 써두기


In [ ]:
df_aug = pd.DataFrame(df_aug)

In [ ]:
# 역번역된 문장의 번호를 이용하여 annotation을 붙인 데이터프레임 만들기
category_list = [] # 순번에 맞춰 category 열 데이터 리스트 만들기
topic_list = [] # 순번에 맞춰 topic 열 데이터 리스트 만들기
sense_list = [] # 순번에 맞춰 sense 열 데이터 리스트 만들기

for i in range(len(df_aug)):
  
  category_list.append(str(df_train['category'][df_aug[1][i]]))
  topic_list.append(str(df_train['topic'][df_aug[1][i]]))
  sense_list.append(str(df_train['sense'][df_aug[1][i]]))


df_aug[2] = category_list # 데이터프레임에 새로운 category 열 만들기
df_aug[3] = topic_list # 데이터프레임에 새로운 topic 열 만들기
df_aug[4] = sense_list # 데이터프레임에 새로운 sense 열 만들기

df_aug = df_aug.drop(1, axis=1) # 더 이상 필요없는 순번 열(1번) 지워주기
df_aug = df_aug.rename(columns = {0:'sentence', 2:'category', 3:'topic', 4:'sense'}) # 각 열의 이름 바꿔주기

In [ ]:
#증강된 데이터를 엑셀로 저장
df_aug.to_excel('.xlsx')